# Modelo de reconocimiento de letras

Por medio del uso del dataset que se encuentra en [keggle](https://www.kaggle.com/datasets/landlord/handwriting-recognition?resource=download), se desea construir un modelo de inteligencia artificial que proporcione la capacidad de analizar entrada de escritura a mano por medio de una entrada por mouse.

In [3]:
import os
import matplotlib as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.keras import utils

DATASET_Jonas = "dataset/archive"

In [10]:
train = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_train_v2.csv"))
train

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE
...,...,...
330956,TRAIN_330957.jpg,LENNY
330957,TRAIN_330958.jpg,TIFFANY
330958,TRAIN_330959.jpg,COUTINHO DESA
330959,TRAIN_330960.jpg,MOURAD


In [9]:
test = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_test_v2.csv"))
test

,0,1
0,FILENAME,IDENTITY
1,TEST_0001.jpg,KEVIN
2,TEST_0002.jpg,CLOTAIRE
3,TEST_0003.jpg,LENA
4,TEST_0004.jpg,JULES
...,...,...
41366,TEST_41366.jpg,ALEXANE
41367,TEST_41367.jpg,PEREIRA-SILVA
41368,TEST_41368.jpg,LAURENT
41369,TEST_41369.jpg,DEFFENSE


In [11]:
validation = pd.read_csv(os.path.join(DATASET_Jonas, "written_name_validation_v2.csv"))
validation

,FILENAME,IDENTITY
0,VALIDATION_0001.jpg,BILEL
1,VALIDATION_0002.jpg,LAUMIONIER
2,VALIDATION_0003.jpg,LEA
3,VALIDATION_0004.jpg,JEAN-ROCH
4,VALIDATION_0005.jpg,RUPP
...,...,...
41365,VALIDATION_41366.jpg,CHAILLAN
41366,VALIDATION_41367.jpg,BAROUH
41367,VALIDATION_41368.jpg,MAXENCE
41368,VALIDATION_41369.jpg,HAMELIN
